In [ ]:
!pip install neuralforecast

In [ ]:
!pip install yahoo-finance

In [ ]:
!pip install ta

In [ ]:
!pip install arch

In [ ]:
from google.colab import files

In [ ]:
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from neuralforecast.models import NBEATSx
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, MSE, MAE, QuantileLoss
from pandas.tseries.offsets import CustomBusinessDay

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import ta as ta
import sklearn as sk
from sklearn import preprocessing
from scipy.stats import t
import tensorflow as tf
from datetime import date, datetime, timedelta
from arch import arch_model
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
import math

In [ ]:
#Return calculation
def ReturnCalculation (Database,lag):
    dimension=Database.shape[0];dif=lag;Out=np.zeros([dimension-dif])
    for i in range(dimension-dif):
        Out[i]=(np.log(Database['Close'][i+dif])-np.log(Database['Close'][i]))
    return np.append(np.repeat(np.nan, dif),Out), Database.index

#STD Calculation
def SDCalculation (DailyReturns, LagSD):
    dimension=DailyReturns.shape[0]; dif=LagSD; Out=np.zeros([dimension-dif])
    for i in range (dimension-dif):
        Out[i]=np.std(DailyReturns[i:i+LagSD],ddof=1)
    return np.append(np.repeat(np.nan, dif),Out)

#STD Calculation
def TrueSDCalculation (DailyReturns, LagSD):
    dimension=DailyReturns.shape[0]; dif=LagSD; Out=np.zeros([dimension-dif+1])
    for i in range (dimension-dif+1):
        Out[i]=np.std(DailyReturns[i:i+LagSD],ddof=1)
    return np.append(Out,np.repeat(np.nan, dif-1))

#Database is calculated
def DatabaseGeneration (Database, Lag, LagSD):
    DailyReturns, Index = ReturnCalculation(Database,Lag)
    DailyReturnsOld =  np.append(np.repeat(np.nan, 1),DailyReturns[0:(DailyReturns.shape[0]-1)])
    SD = SDCalculation (DailyReturns, LagSD)
    TrueSD = TrueSDCalculation(DailyReturns, LagSD)
    Data = pd.DataFrame({'DailyReturns': DailyReturns, 'SD': SD, 'TrueSD': TrueSD, 'DailyReturnsOld': DailyReturnsOld})
    Data = Data.set_index(Index)
    return Data.dropna()

#Database is calculated
def DatabaseGeneration_NBEATS (Database, Lag, LagSD):
    DailyReturns, Index = ReturnCalculation(Database,Lag)
    DailyReturnsOld =  np.append(np.repeat(np.nan, 1),DailyReturns[0:(DailyReturns.shape[0]-1)])
    SD = SDCalculation (DailyReturns, LagSD)
    TrueSD = TrueSDCalculation(DailyReturns, LagSD)
    Data = pd.DataFrame({'Date': Database.index,'DailyReturns': DailyReturns, 'SD': SD, 'TrueSD': TrueSD, 'DailyReturnsOld': DailyReturnsOld})
    Data = Data.set_index(Index)
    return Data.dropna()

In [ ]:
#MultiHeadSelfAttention
class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}")
        self.projection_dim = embed_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embed_dim)
        self.key_dense = tf.keras.layers.Dense(embed_dim)
        self.value_dense = tf.keras.layers.Dense(embed_dim)
        self.combine_heads = tf.keras.layers.Dense(embed_dim)
    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights
    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(query, batch_size)  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(key, batch_size)  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(value, batch_size)  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(concat_attention)  # (batch_size, seq_len, embed_dim)
        return output

#Transformer Keras Block
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        # self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.nb_dict = {}; self.Bagging=5
        for i in range(self.Bagging):
          self.nb_dict["att{0}".format(i)]=MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = tf.keras.Sequential([tf.keras.layers.Dense(ff_dim, activation="relu"), tf.keras.layers.Dense(embed_dim),])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
    def call(self, inputs, training):
        self.att_dict = {}
        for i in range(self.Bagging):
          self.att_dict["att{0}".format(i)]=self.nb_dict["att{0}".format(i)](tf.keras.layers.Dropout(.1)(inputs))
          if i==0:
            self.att_dict["attn_output"]=self.att_dict["att{0}".format(i)]/self.Bagging
          else:
            self.att_dict["attn_output"]=self.att_dict["attn_output"]+self.att_dict["att{0}".format(i)]/self.Bagging
        attn_output = self.dropout1(self.att_dict["attn_output"], training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

def Transformer_Model (Shape1, Shape2, HeadsAttention,Dropout, LearningRate):
    #Model struture is defined
    Input = tf.keras.Input(shape=(Shape1,Shape2), name="Input")
    #LSTM is applied on top of the transformer
    X = tf.keras.layers.LSTM(units=16, dropout=Dropout, return_sequences=True)(Input)
    #Tranformer architecture is implemented
    transformer_block_1 = TransformerBlock(embed_dim=16, num_heads=HeadsAttention, ff_dim=8, rate=Dropout)
    X = transformer_block_1(X)
    #Dense layers are used
    X = tf.keras.layers.GlobalAveragePooling1D()(X)
    X = tf.keras.layers.Dense(8, activation=tf.nn.sigmoid)(X)
    X = tf.keras.layers.Dropout(Dropout)(X)
    Output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid, name="Output")(X)
    model = tf.keras.Model(inputs=Input, outputs=Output)
    #Optimizer is defined
    Opt = tf.keras.optimizers.Adam(learning_rate=LearningRate, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
    #Model is compiled
    model.compile(optimizer=Opt, loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

In [ ]:
#It generates the database for fitting transformer. No positional encoding is needed as LSTM plays this role in the model structure
def Transformer_Database (Timestep, XData_AR, YData_AR):
    Features = XData_AR.shape[1]; Sample = XData_AR.shape[0]-Timestep+1
    XDataTrainScaledRNN=np.zeros([Sample, Timestep, Features]); YDataTrainRNN=np.zeros([Sample])
    for i in range(Sample):
        XDataTrainScaledRNN[i,:,:] = XData_AR[i:(Timestep+i)]
        YDataTrainRNN[i] = YData_AR[Timestep+i-1]
    return XDataTrainScaledRNN, YDataTrainRNN

#Database is calculated
def DatabaseGenerationForecast (Database, Lag, LagSD):
    DailyReturns, Index = ReturnCalculation(Database,Lag)
    DailyReturnsOld =  np.append(np.repeat(np.nan, 1),DailyReturns[0:(DailyReturns.shape[0]-1)])
    SD = SDCalculation (DailyReturns, LagSD)
    TrueSD = TrueSDCalculation(DailyReturns, LagSD)
    Data = pd.DataFrame({'DailyReturns': DailyReturns, 'SD': SD, 'TrueSD': TrueSD, 'DailyReturnsOld': DailyReturnsOld})
    Data = Data.set_index(Index)
    return Data

In [ ]:
#Final AR database for forcasting is generated
def DatabaseGenerationForecast_AR (Database, Lag, LagSD, forecast_nbeats):
    Data_Forecast=DatabaseGenerationForecast(Database, Lag, LagSD).iloc[(-LagSD+1)]
    Index_Forecast=DatabaseGenerationForecast(Database, Lag, LagSD).index[(-LagSD+1)]
    XDataForecast={'SD': Data_Forecast['SD'], 'DailyReturnsOld': Data_Forecast['DailyReturnsOld'],
               'NBEATS-median' : forecast_nbeats}
    return pd.DataFrame([XDataForecast], index=[Index_Forecast]), Data_Forecast['DailyReturns']

#Transformed ANN-ARCH model forecast
def T_ANN_ARCH_Forecast (Database, Lag, LagSD, forecast_nbeats, Scaled_Norm, XData_AR, model):
    XDataForecast, ReturnForecast = DatabaseGenerationForecast_AR (Database, Lag, LagSD, forecast_nbeats)
    XDataForecast = pd.concat([XData_AR,XDataForecast])
    XDataForecastTotalScaled = Scaled_Norm.transform(XDataForecast)
    XDataForecastTotalScaled_T, Y_T = Transformer_Database(Timestep, XDataForecastTotalScaled, np.zeros(XDataForecastTotalScaled.shape[0]))
    TransformerPrediction = model.predict(XDataForecastTotalScaled_T)
    return TransformerPrediction[-1][0], XDataForecast.index[-1], TransformerPrediction[0:(XDataForecastTotalScaled_T.shape[0]-1)], ReturnForecast

#It calculates VaR taking into consideration the forecasted sigma to calculate the scale parameter
def T_ANN_ARCH_VaR (Alpha, HistoricalReturns, ForecastedSigma, DF):
    HistoricalMean = np.mean(HistoricalReturns)
    ScaleParameter = np.sqrt((ForecastedSigma**2)*((DF-2)/DF))
    VaR = -t.ppf(Alpha, DF, loc=HistoricalMean, scale=ScaleParameter)
    return VaR

In [ ]:
def nbeats_model_forecasts(Data):
  start=Data.index[0].date()
  df = yf.download("^CRSLDX", start = '2000-01-01', end = start+ timedelta(days=4))
  df = df.resample('D').fillna(method='ffill')
  temp_df = DatabaseGeneration_NBEATS(df, 1, 5)
  temp_df['Date'] = temp_df['Date'].map(lambda x: x.date())
  if temp_df['Date'].iloc[-1] < start-timedelta(days=1):

    new_date_range = pd.date_range(start=temp_df.index.min(), end= start-timedelta(days=1))

    # Reindex the DataFrame to the new date range, forward-filling missing values
    temp_df = temp_df.reindex(new_date_range).ffill()

    # Reset the index if you want 'date' back as a column
    temp_df.reset_index(inplace=True)
    temp_df = temp_df.drop('Date', axis=1)
    temp_df.rename(columns={'index': 'Date'}, inplace=True)

  temp_df['unique_id'] = '1'
  temp_df.rename(columns={'Date':'ds'}, inplace=True)
  temp_df.rename(columns={'TrueSD':'y'}, inplace=True)

  model = NBEATS(h=len(Data), input_size=1,
                loss=MQLoss(level=[90]),
                learning_rate= 0.001,
                stack_types=['identity', 'identity'],
                n_blocks=[1,1],
                mlp_units= [[512, 512], [512, 512]],
                # windows_batch_size=60,
                num_lr_decays=5,
                # val_check_steps=20,
                n_harmonics=0, n_polynomials=0,

                max_steps=200,
                # early_stop_patience_steps=2,

                )
  fcst = NeuralForecast(
    models=[model],
    freq='D'
  )

  temp_df['ds'] = temp_df['ds'].apply(lambda x: pd.Timestamp(x))

  fcst.fit(df=temp_df)
  forecasts = fcst.predict()
  return forecasts


In [ ]:
def nbeats_nextday_forecast(Data):

  Data['unique_id'] = '1'
  Data.rename(columns={'Date':'ds'}, inplace=True)
  Data.rename(columns={'TrueSD':'y'}, inplace=True)

  model = NBEATS(h=1, input_size=1,
                loss=MQLoss(level=[90]),
                learning_rate= 0.001,
                stack_types=['identity', 'identity'],
                n_blocks=[1,1],
                mlp_units= [[512, 512], [512, 512]],
                # windows_batch_size=60,
                num_lr_decays=5,
                # val_check_steps=20,
                n_harmonics=0, n_polynomials=0,

                max_steps=500,
                # early_stop_patience_steps=2,

                )
  fcst = NeuralForecast(
    models=[model],
    freq='D'
  )

  fcst.fit(df=Data)
  forecasts = fcst.predict()
  return forecasts


In [ ]:
#Fitting of Transformed ANN-ARCH model and forecasting of the next volatility value
def T_ANN_ARCH_Fit (Data, Lag=1, LagSD=5, Timestep=10, Dropout=0.05, LearningRate=0.01, Epochs=10000, Alpha=0.005, DF=4, BatchSize=64):
    #N-Beats Model is fitted
    pred_nbeats = nbeats_model_forecasts(Data)
    nbeats_nextday = nbeats_nextday_forecast(Data)

    pred_nbeats.rename(columns={'ds': 'Date'}, inplace=True)
    pred_nbeats = pred_nbeats.set_index('Date')

    Data = Data.drop('ds', axis=1)
    Data.rename(columns={'y': 'TrueSD'}, inplace=True)
    Data = Data.drop('unique_id', axis=1)
    # Database contaning N-Beats model is generated
    Data_AR=pd.concat([Data, pred_nbeats['NBEATS-median']], axis=1)
    if Data_AR.shape[0]!=Data.shape[0]: print("Error in DB Generation")
    #Original explanatory and response variables are generated
    XData_AR = Data_AR.drop(Data_AR.columns[[0,2]], axis=1);YData_AR = Data_AR['TrueSD']
    #Data is normalized
    Scaled_Norm = preprocessing.StandardScaler().fit(XData_AR); XData_AR_Norm = Scaled_Norm.transform(XData_AR)
    #Data for fitting the transformer model is generated
    XData_AR_Norm_T, YData_AR_Norm_T= Transformer_Database(Timestep, XData_AR_Norm, YData_AR)
    #Model with transformer layer is defined
    model = Transformer_Model(XData_AR_Norm_T.shape[1], XData_AR_Norm_T.shape[2], HeadsAttention=4, Dropout=Dropout, LearningRate=LearningRate)
    model.fit(XData_AR_Norm_T, YData_AR_Norm_T, epochs=Epochs, verbose=0, batch_size=BatchSize); tf.keras.backend.clear_session()
    Forecast, Date_Forecast, TrainPrediction, ReturnForecast = T_ANN_ARCH_Forecast (Database, Lag, LagSD, nbeats_nextday['NBEATS-median'].values.item(0), Scaled_Norm, XData_AR, model)
    VaR = T_ANN_ARCH_VaR(Alpha, Data['DailyReturnsOld'], Forecast,DF)
    return {'T_ANN_ARCH_model':model, 'Forecast_T_ANN_ARCH':Forecast, 'Date_Forecast':Date_Forecast, 'TrainPrediction': TrainPrediction, 'Scaler':Scaled_Norm,  'ReturnForecast':ReturnForecast, 'Forecast_NBEATS': nbeats_nextday['NBEATS-median'].values.item(0)  , 'YData_Train':YData_AR_Norm_T, 'VaR': VaR}

In [ ]:
#Index of end dates, database for validation and dataframe to collect the results are created. Model variables are defined.
Start='2008-01-01'; End='2016-12-31'; IndexEndDays=yf.download("^CRSLDX",start=Start,  end=End, progress=False).index
Lag=1; LagSD=5; Timestep=10; Dropout=0.15; LearningRate=0.01; Epochs=50; Alpha=0.005; DF=4
validation_df = yf.download("^GSPC",start='2000-01-01', end=date.today()+timedelta(days=1), progress=False)
validation_df = validation_df.resample('D').fillna(method='ffill')
DataValidation = DatabaseGeneration(validation_df, Lag, LagSD)
ResultsCollection=pd.DataFrame({'Date_Forecast': [], 'Forecast_T_ANN_ARCH': [], 'ReturnForecast':[],'TrueSD':[], 'VaR_T_ANN_ARCH':[], 'Forecast_NBEATS':[]})


In [ ]:
#Loop for generating the results
for i in tqdm(range(IndexEndDays.shape[0])):
    #Database is downloaded from yahoo finance and lag of returns defined
    Database=yf.download("^CRSLDX",start=IndexEndDays[i].date()-timedelta(days=650), end=IndexEndDays[i].date(), progress=False)

    Database = Database.resample('D').fillna(method='ffill')
    # Database.index = pd.to_datetime(Database.index)
    Database.index = Database.index.map(lambda x: x.date())
    if Database.index[-1] < IndexEndDays[i].date()-timedelta(days=1):

      new_date_range = pd.date_range(start=Database.index.min(), end= IndexEndDays[i].date()-timedelta(days=1))

      # Reindex the DataFrame to the new date range, forward-filling missing values
      Database = Database.reindex(new_date_range).ffill()

      # Reset the index if you want 'date' back as a column
      Database.reset_index(inplace=True)
      # Database = Database.drop('Date', axis=1)
      Database.rename(columns={'index': 'Date'}, inplace=True)

      Database = Database.set_index('Date')
      Database.index = pd.to_datetime(Database.index)
    print(Database)
    Database.index = Database.index.map(lambda x: pd.Timestamp(x))
    #Database for fitting the models is generated
    Data = DatabaseGeneration_NBEATS(Database, Lag, LagSD)
    T_ANN_ARCH_Model = T_ANN_ARCH_Fit (Data, Lag, LagSD, Timestep, Dropout, LearningRate, Epochs, Alpha, DF)
    # Results are collected
    IterResults={'Date_Forecast': T_ANN_ARCH_Model['Date_Forecast'].date(), 'Forecast_T_ANN_ARCH': T_ANN_ARCH_Model['Forecast_T_ANN_ARCH'],'Forecast_NBEATS':T_ANN_ARCH_Model['Forecast_NBEATS'],'ReturnForecast':T_ANN_ARCH_Model['ReturnForecast'],'TrueSD':DataValidation[DataValidation.index==pd.to_datetime(T_ANN_ARCH_Model['Date_Forecast'].date())]['TrueSD'][0], 'VaR_T_ANN_ARCH': T_ANN_ARCH_Model['VaR']}
    ResultsCollection=ResultsCollection._append(IterResults, ignore_index=True)
    # Results are saved
    ResultsCollection.to_csv('Nifty_Nbeats_transformers_train.csv',index=False)